In [2]:
import yaml
import logging
import jsonschema
import asyncio

from lsst.ts import salobj
from lsst.ts.observatory.control.auxtel import ATQueue
from lsst.ts.idl.enums.ScriptQueue import Location, SalIndex
from lsst.ts.observatory.control.auxtel.atcs import ATCS
from lsst.ts.observatory.control.utils import RotType

## Start Auxiliary Telescope

In [4]:
domain = salobj.Domain()
atq = ATQueue(domain = domain)
await atq.start_task

[None]

In [5]:
atcs = ATCS(domain)
await atcs.start_task

[None, None, None, None, None, None, None]

In [7]:
await atcs.open_m1_cover()
await atcs.open_m1_vent()

In [8]:
await atcs.enable_ataos_corrections()

In [10]:
await atcs.rem.ataos.cmd_resetOffset.set_start(axis = "z")

## Align telescope

In [7]:
script_path = "auxtel/latiss_wep_align.py"
band_filter = 'g'

script_config = dict(
    track_target = dict(
        target_name = 'DUN 149',
        icrs = dict(
            ra = 13+53/60+32.71/60/60,
            dec = -38+15/60+57.7/60/60
        )
    ),
    filter = f"SDSS{band_filter}_65mm", 
    grating = 'empty_1',
    exposure_time = 30,
    reason = 'AOS_align',
    program = 'SITCOM-682'
)
'''
track_target = dict(
    target_name = 'MWSC 1728',
    icrs = dict(
        ra = 148.867*24/360,
        dec = -28.965
    )
),
'''
script_description=f"Testing SITCOM-682 alignment"

await atq.validate_config(is_standard=False, script=script_path, config=script_config)

await atq.add_external(script=script_path, config=script_config, description=script_description)

## SITCOM-681 Sensitivity matrix


In [54]:
offsets = {
    "x": np.linspace(-2.0,2.0, 10),
    "y": np.linspace(-2.0,2.0, 10),
    "z": np.linspace(-0.1,0.1, 10),
    "rx": np.linspace(-15.0,15.0,10)/60,
    "ry": np.linspace(-15.0,15.0,10)/60,
    "m1": np.linspace(-12000,-4000,10),
}

In [59]:
script_path = "auxtel/latiss_intra_extra_focal_data.py"
band_filter = 'r'
exposure_time = 20

for var, offset_list in offsets.items():
    print(var)
    for offset in offset_list:
        script_config = {
            'filter': f"SDSS{band_filter}_65mm", 
            'grating': 'empty_1',
            'exposure_time': exposure_time,
            f'offset_{var}': float(offset),
            'reason': f'AOS_SM_offset_{var}_{offset:.3f}',
            'program': 'SITCOM-681',
        }
        script_description=f"Testing SITCOM-681 offset_{var} {offset}"

        await atq.validate_config(is_standard=False, script=script_path, config=script_config)

        await atq.add_external(script=script_path, config=script_config, description=script_description)

x


y


z


rx


ry


m1


In [43]:
script_path = "auxtel/latiss_intra_extra_focal_data.py"
band_filter = 'r'
exposure_time = 20

script_config = dict(
    filter = f"SDSS{band_filter}_65mm", 
    grating = 'empty_1',
    take_detection_image = True,
    exposure_time = 20,
    reason = f'AOS_pointing_no_offset',
    program = 'SITCOM-680'
)
script_description=f"Testing SITCOM-680 pointing no offset"

await atq.validate_config(is_standard=False, script=script_path, config=script_config)

await atq.add_external(script=script_path, config=script_config, description=script_description)

## SITCOM-684 Medium density fields


In [ ]:
target_labels = ['MWSC 1728', 'MWSC 1497', 'MWSC 1300', 'MWSC 1159', 'MWSC 1151']
target_ra = [148.867, 127.800, 114.922, 109.252, 108.625]
target_dec = [-28.965, -44.505, -27.295, -17.155, -21.123]

In [ ]:
script_path = "auxtel/latiss_intra_extra_focal_data.py"
band_filter = 'r'

for idx, target_name in enumerate(target_labels):

    script_config = dict(
        track_target = dict(
            target_name = target_name,
            icrs = dict(
                ra = target_ra[idx]*24/360,
                dec = target_dec[idx]
            )
        )
        filter = f"SDSS{band_filter}_65mm", 
        grating = 'empty_1',
        exposure_time = 20,
        reason = 'AOS_medium_density',
        program = 'SITCOM-684'
    )
    script_description=f"Testing SITCOM-684 Medium density field analysis"
    
    await atq.validate_config(is_standard=False, script=script_path, config=script_config)

    await atq.add_external(script=script_path, config=script_config, description=script_description)

## SITCOM-683 Giant Donuts


In [ ]:
script_path = "auxtel/latiss_intra_extra_focal_data.py"
band_filter = 'r'
dz = 0.8 #default mm

script_config = dict(
    filter = f"SDSS{band_filter}_65mm", 
    grating = 'empty_1',
    dz = dz,
    reason = 'AOS_Giant_Donut',
    program = 'SITCOM-683'
)
script_description=f"Testing SITCOM-683 Giant Donuts, giant donut dz {dz} mm"

await atq.validate_config(is_standard=False, script=script_path, config=script_config)

await atq.add_external(script=script_path, config=script_config, description=script_description)

## SITCOM-685 Exposure time

In [57]:
script_path = "auxtel/latiss_intra_extra_focal_data.py"
band_filter = 'r'
exposure_times = [20, 40, 60, 80, 100, 120, 160] #default s

for exposure_time in exposure_times:
    script_config = dict(
        filter = f"SDSS{band_filter}_65mm", 
        grating = 'empty_1',
        exposure_time = exposure_time,
        reason = f'AOS_Exposure_time_{exposure_time}',
        program = 'SITCOM-685'
    )
    script_description=f"Testing SITCOM-685 exposure time {exposure_time}"

    await atq.validate_config(is_standard=False, script=script_path, config=script_config)

    await atq.add_external(script=script_path, config=script_config, description=script_description)

## SITCOM-682 Double stars

In [ ]:
targets = ['STF 956', 'S 563', 'DUN 71', 'HJ 4299', 'DUN 149', 'HJ 561']


06h42m40.55s   +01d42m58.2s 
08h08m29.64s   -19d51m36.8s
08h30m34.41s   -40d30m42.0s
10h13m21.19s   -50d54m27.2s
13h53m32.71s   -38d15m57.7s
14h56m37.90s   -14d06m14.7s

ra = 14+56/60+37.90/60/60,
dec = -14-6/60-14.7/60/60

In [21]:
script_path = "auxtel/latiss_intra_extra_focal_data.py"
band_filter = 'r'
dz = 6 #default mm

script_config = dict(
    dz = dz,
    filter = f"SDSS{band_filter}_65mm", 
    grating = 'empty_1',
    reason = f'AOS_Giant_Donut_HJ_561_dz_{dz}mm',
    program = 'SITCOM-683'
)
script_description=f"Testing SITCOM-683 Giant Donuts"

await atq.validate_config(is_standard=False, script=script_path, config=script_config)

await atq.add_external(script=script_path, config=script_config, description=script_description)

In [15]:
ra = 13+53/60+32.71/60/60,
dec = -38-15/60-57.7/60/60

13.892419444444444

In [16]:
-38-15/60-57.7/60/60

-38.26602777777778